In [260]:
#!pip install transformers

In [261]:
#!pip install sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

2021-11-25 16:38:03.563273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/apps/cuDNN/7.6.5-cuda-10.2/lib64
2021-11-25 16:38:03.563307: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [3]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [4]:
import pandas as pd
citants = pd.read_json('From-ScisummNet-2019/A00-1031/citing_sentences.json')

In [5]:
citants = citants[['citance_No','clean_text']]
citants
queries = list(citants['clean_text'])
# cit_no = list(citants['citance_No'])
# query_sent = citants['raw_text']
# query_sent

In [6]:
cite_no = list(citants.citance_No)

In [7]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse("From-ScisummNet-2019/A00-1031"+"/Reference_XML/"+"A00-1031"+".xml")
root = tree.getroot()
final1=[]
final2=[]
i = 0
total = len(root)
# print(total)
for a in root:
  #  print(a.attrib)
   # print(i)
    for b in a:

        final1.append(b.text)
        if i == 0:
            final2.append("Abstract")
        if i == 1:
            final2.append("Introduction")
        elif i < total-2:
            final2.append("Experiment/Discussion")
        if i == total-2 or i == total-1:
            final2.append("Results/Conclusion")
        if i == total:
            final2.append("Acknowledgment")
    i = i+1


d={'col1':final1,'col2':final2}
rp = pd.DataFrame(data=d)

In [8]:
rp.head

<bound method NDFrame.head of                                                   col1                col2
0    Trigrams'n'Tags (TnT) is an efficient statisti...        Introduction
1    Contrary to claims found elsewhere in the lite...        Introduction
2    A recent comparison has even shown that TnT pe...        Introduction
3    We describe the basic model of TnT, the techni...        Introduction
4    Furthermore, we present evaluations on two cor...        Introduction
..                                                 ...                 ...
173  Many thanks go to Hans Uszkoreit for his suppo...  Results/Conclusion
174  Most of the work on TnT was carried out while ...  Results/Conclusion
175  Large annotated corpora are the pre-requisite ...  Results/Conclusion
176  Therefore, I would like to thank all the peopl...  Results/Conclusion
177  And, last but not least, I would like to thank...  Results/Conclusion

[178 rows x 2 columns]>

In [9]:
rp.col1

0      Trigrams'n'Tags (TnT) is an efficient statisti...
1      Contrary to claims found elsewhere in the lite...
2      A recent comparison has even shown that TnT pe...
3      We describe the basic model of TnT, the techni...
4      Furthermore, we present evaluations on two cor...
                             ...                        
173    Many thanks go to Hans Uszkoreit for his suppo...
174    Most of the work on TnT was carried out while ...
175    Large annotated corpora are the pre-requisite ...
176    Therefore, I would like to thank all the peopl...
177    And, last but not least, I would like to thank...
Name: col1, Length: 178, dtype: object

In [10]:
ann = "From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt"
with open(ann,"r") as f:
    data = f.read()

In [11]:
idx = {'Citance Number':0,'Reference Article':1,'Citing Article':2,'Citation Marker Offset':3,'Citation Marker':4,'Citation Offset':5,'Reference Offset':7}

In [12]:
gt = {}
lines = data.split('\n')
for line in lines:
    lis = line.split('|')
#     print(lis, len(lis))
    if len(lis)==11:
            cit_no = lis[0].split(':')[1]
            ref_off = lis[7].split(':')[1].strip()
            exec("ref_off="+ref_off)
            ref_off = list(map(int, ref_off))
            gt[int(cit_no)] = ref_off #

In [13]:
gt

{1: [0, 36],
 2: [44, 178],
 3: [15, 173],
 4: [0, 1],
 5: [27, 165],
 6: [27, 141],
 7: [115, 154],
 8: [24, 165],
 9: [36, 46],
 10: [2, 75],
 11: [0, 1],
 12: [18, 40],
 13: [36, 118],
 14: [24, 173],
 15: [79, 80],
 16: [0, 113],
 17: [12, 19],
 18: [0, 1],
 19: [28, 157],
 20: [36, 164]}

In [14]:
# def clean(line):
#     line = line.split("|")
#     cit_2 , ref_2 = line[6].split("Citation Text:")[1].strip() , line[8].split("Reference Text:")[1].strip()
#     cit_2 = cit_2.split(" >")[1].split("</S>")[0]
#     ref_2 = ref_2.split(" >")[1:]
#     ref_2 = [ i.split("</S>")[0] for i in ref_2]
#     ans=""
#     for i in ref_2:
#         ans=ans + cit_2 + " $$$$$ " + i +"\n"
#    # ref_2 = " ".join(ref_2)
    
#     return ans

In [15]:
# with open('./From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt') as f:
#     data = f.read()
# data = [ clean(i) for i in data.split("\n") if i ]

# # with open("cleaned.txt","w+") as f:
# #     f.write("\n".join(data))

In [16]:
corpus = rp.col1

In [17]:
cite_no

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [19]:
import scipy
import numpy as np

In [30]:
def get_matching_sentences(model):
    # Get a vector for each headline (sentence) in the corpus
    corpus_embeddings = model.encode(corpus)
    # Define search queries and embed them to vectors as well

    query_embeddings = model.encode(queries)
    # For each search term return 5 closest sentences
    closest_n = 5

    for i in range(len(queries)):
        query, query_embedding  = queries[i], query_embeddings[i]
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    #     results = dict(enumerate(distances,1))
    #     results = dict(sorted(results.items(), key=lambda item: item[1]))

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")
        indexes = results[0:closest_n]
        top_n = []
        for l,k in indexes:
            top_n.append(l)
        print(np.intersect1d(top_n,gt[cite_no[i]]))
        print("indexes{}, top_n{}, i{}, cite_no[i]{} ,gt[cite_no[i]]{}".format(indexes, top_n,i,cite_no[i],gt[cite_no[i]]))

    #     for idx, distance in results[0:closest_n]:
    #         print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))


In [32]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# #Define your train examples. You need more than just two examples...

# train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#     InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

# #Define your train dataset, the dataloader and the train loss
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.CosineSimilarityLoss(model)

# #Tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100)

In [33]:
data = []
for i in range(len(queries)):
    for j in gt[cite_no[i]]:
        if j < len(corpus):
#             print(queries[i],gt[cite_no[i]],corpus[j])
            data.append(InputExample(texts=[queries[i],corpus[j]],label=0.8))
    
        

In [37]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10000, warmup_steps=100)

Epoch:   0%|          | 0/10000 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
get_matching_sentences(model)





Query: The sentences in the DSO collection were tagged with parts of speech using TnT (Brants, 2000) trained on the Brown Corpus itself.

Top 5 most similar sentences in corpus:
[]
indexes[(16, 0.16928802112238228), (174, 0.18503252078064458), (3, 0.20450969856950163), (22, 0.22050259062072064), (26, 0.2350300655004096)], top_n[16, 174, 3, 22, 26], i0, cite_no[i]1 ,gt[cite_no[i]][0, 36]




Query: 

Top 5 most similar sentences in corpus:
[]
indexes[(34, 0.25048548476019006), (70, 0.274554260262443), (57, 0.304607509508648), (4, 0.3228911829779897), (35, 0.33645642478442295)], top_n[34, 70, 57, 4, 35], i1, cite_no[i]2 ,gt[cite_no[i]][44, 178]




Query: The English POS-tagging has been carried out using freely available TNT tagger (Brants, 2000).

Top 5 most similar sentences in corpus:
[]
indexes[(3, 0.20435609224374862), (16, 0.2369332350313279), (74, 0.24585787314736807), (0, 0.2461844095544632), (174, 0.24781550721566248)], top_n[3, 16, 74, 0, 174], i2, cite_no[i]3 ,gt[cite_no[